In [1]:
# Phcli Jupyter Python Template
# 
# 使用手册：
# 1. 请将全局变量定义在第一个输入区内
# 2. Phcli 会自动在第二个输入区初始化 Spark Session
# 3. 所有 print 会在 phcli maxauto dag 后自动转为 logger.debug() 方法
# 4. 请在第三个输入区开始编码，phcli maxauto dag 后会全部归类为一个方法


# Config defined in here

############## == config == ###################
job_name = "clean_detection_data"
job_runtime = "python3"
job_command = "submit"
job_timeout = 720.0
############## == config == ###################


# Variables defined in here

############## == input args == ###################
g_input_paramater="Empty"
############## == input args == ###################

############## == output args == ###################
g_out_parameter="Empty"
############## == output args == ###################

############## == preset function == ###################
from phcli.ph_max_auto.ph_hook.get_abs_path import get_result_path
from phcli.ph_max_auto.ph_hook.get_abs_path import get_depends_path
# result_path_prefix = get_result_path({"name":job_name})
# depends_path = get_depends_path({"name":job_name})
############## == preset function == ###################


In [2]:
from pyspark.sql import  SparkSession    
spark = SparkSession.builder \
    .master("yarn") \
    .appName("hadoop clean_detection_data in jupyter using python3") \
    .config("spark.driver.cores", "1") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.instances", "1") \
    .config('spark.sql.codegen.wholeStage', False) \
    .config('spark.sql.execution.arrow.pyspark.enabled', True) \
    .getOrCreate()

In [3]:
from pyspark.sql.types import IntegerType, DoubleType, StringType, StructType
from pyspark.sql.functions import col, date_format, count, isnull, lit
from pyspark.sql.functions import when, isnan, udf, pandas_udf, PandasUDFType
from pyspark.sql.window import Window
from pyspark.sql import functions as Func
from pyspark.sql import DataFrame, SparkSession    

from typing import Iterator

import pandas as pd
import re

In [ ]:
# %%

## 参数化文件读入

In [4]:
# %%
## ====== 输入文件和输出文件 ======

# 
g_whether_save_result = True

p_main_dir = "s3://ph-origin-files/user/zazhao/2020年结果-csv/"

# 检测数据
p_detection = p_main_dir + "检测"


# # 输入目录
# p_patient_std_dir = "s3://ph-origin-files/user/zazhao/2020年结果-csv/HIS_result/clean_data_result"

# # 输出文件
# p_patient_std_correlation_out = p_main_dir+"HIS_result/" + "correlation_data_result"

In [5]:
# %%

## 读取检测数据
df_raw_detection = spark.read.csv(p_detection, header=True)
# df_raw_detection.show(1)

df_raw_detection = df_raw_detection.select([ 'PATIENT_ID', 'VISIT_ID', 'ITEM_NAME', 'SUBJECT', 'REPORT_ITEM_NAME', 
                                            'RESULT', 'UNITS', 'ABNORMAL_INDICATOR', 'REQUESTED_DATE_TIME', 
                                            'RESULTS_RPT_DATE_TIME', 'DEPT_NAME'])

df_raw_detection = df_raw_detection.withColumn("VISIT_ID", Func.col("VISIT_ID").cast("int"))\
                                    .withColumn("REQUESTED_DATE_TIME_STD",  date_format("REQUESTED_DATE_TIME", "yyyyMMdd")) \
                                    .withColumn("RESULTS_RPT_DATE_TIME_STD", date_format("RESULTS_RPT_DATE_TIME", "yyyyMMdd"))\
                                    .withColumn("检测信息", Func.concat( col("REPORT_ITEM_NAME"), col("RESULT"), col("UNITS")) )
# df_raw_detection.where( df_raw_detection["VISIT_ID"].isNull() ).count()

In [9]:
body_info = [["Bom", "20", 97.6, 165],
             ["Alice", "23", 90.0, 160],
             ["Alice", "aa", 99.0, 92],
             ]

body_df = spark.createDataFrame(body_info, ["name", "age", "weight", "height"])
body_df.show()

+-----+---+------+------+
| name|age|weight|height|
+-----+---+------+------+
|  Bom| 20|  97.6|   165|
|Alice| 23|  90.0|   160|
|Alice| aa|  99.0|    92|
+-----+---+------+------+



In [19]:
from pyspark.sql.types import IntegerType

# body_df.withColumn("cc", col("age").cast("int")).show()
# body_df.withColumn("cc", col("age").cast( "float" ))\
#         .withColumn("cc", Func.when( col("cc").isNotNull(), col("cc") ).otherwise( col("age")) )\
#         .show()

body_df.withColumn("cc", Func.when( col("age").cast( "float" ).isNotNull(), "number" ).otherwise( "string" ) )\
        .show()

+-----+---+------+------+------+
| name|age|weight|height|    cc|
+-----+---+------+------+------+
|  Bom| 20|  97.6|   165|number|
|Alice| 23|  90.0|   160|number|
|Alice| aa|  99.0|    92|string|
+-----+---+------+------+------+



In [6]:
df_raw_detection.show(1, vertical=True)
# df_raw_detection.select("ITEM_NAME").distinct().show(50)

-RECORD 0-----------------------------------------
 PATIENT_ID                | 92f04cbea43683007... 
 VISIT_ID                  | 1                    
 ITEM_NAME                 | 全血细胞分析YG       
 SUBJECT                   | 全血细胞分析YG       
 REPORT_ITEM_NAME          | 快速C反应蛋白        
 RESULT                    | 1.0                  
 UNITS                     | mg/L                 
 ABNORMAL_INDICATOR        | N                    
 REQUESTED_DATE_TIME       | 2020-08-26 10:21:... 
 RESULTS_RPT_DATE_TIME     | 2020-08-27 07:44:... 
 DEPT_NAME                 | 新生儿科四病区       
 REQUESTED_DATE_TIME_STD   | 20200826             
 RESULTS_RPT_DATE_TIME_STD | 20200827             
 检测信息                  | 快速C反应蛋白1.0mg/L 
only showing top 1 row



In [26]:
df_raw_detection = df_raw_detection.withColumn("result_type", Func.when(  col("RESULT").cast( "float" ).isNotNull(),
                                                                        "numbers_only" ).otherwise( "character" ))
df_raw_detection = df_raw_detection.withColumn("lab_result",  Func.when(  col("RESULT").cast( "float" ).isNotNull(),
                                                                         col("RESULT").cast( "float" ) ).otherwise( col("RESULT") ))


df_raw_detection = df_raw_detection.withColumn("std_subject", Func.when( col("REPORT_ITEM_NAME").rlike('降钙素|PCT检测'),'降钙素原' )\
                                                                  .when( col("REPORT_ITEM_NAME").rlike('白细胞计数'),'白细胞计数'  )\
                                                                  .when( col("REPORT_ITEM_NAME").rlike('反应蛋白'),'C反应蛋白'  )\
                                                                  .when( col("REPORT_ITEM_NAME").rlike('菌|病毒|支原体|衣原体|结核'),'病原菌检测'  )\
                                                                  .otherwise( '其他检测项目') )


# df_raw_detection =df_raw_detection.filter( col("std_subject")=="病原菌检测")

df_raw_detection = df_raw_detection.withColumn("pathogenic_type", 
                                    Func.when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("嗜肺")  ) , "嗜肺军团菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("肺炎衣原体")  ) , "肺炎衣原体" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("肺炎支原体")  ) , "肺炎支原体" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("冠状病毒")  ) , "冠状病毒" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("合胞病毒")  ) , "合胞病毒" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("流感病毒")  ) , "流感病毒" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("腺病毒")  ) , "腺病毒" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("柯萨奇病毒")  ) , "柯萨奇病毒" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("鲍曼氏不动")  ) , "鲍曼氏不动" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("大肠埃希菌")  ) , "大肠埃希菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("克雷伯")  ) , "肺炎克雷伯菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("肺炎链球菌")  ) , "肺炎链球菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("金黄色葡萄球菌")  ) , "金黄色葡萄球菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("流感嗜血菌")  ) , "流感嗜血菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("嗜麦芽寡养")  ) , "嗜麦芽寡养单胞菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("嗜麦芽窄食")  ) , "嗜麦芽窄食单胞菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("铜绿")  ) , "铜绿假单胞菌" )\
                                            .when( ( col("std_subject")=="病原菌检测") &( col("REPORT_ITEM_NAME").rlike("阴沟肠杆菌")  ) , "阴沟肠杆菌" )\
                                            .otherwise("未知") )
## 还有其他的细胞病毒
## 巨细胞病毒


# value_range 是什么？

df_raw_detection = df_raw_detection.withColumn("std_result", Func.when( ( col("std_subject")=="降钙素原") &( col("result_type")=="numbers_only" )&\
                                                                            ( col("lab_result")<0.1), '<0.1(无感染)' )\
                                                                   .when( ( col("std_subject")=="降钙素原") &( col("result_type")=="numbers_only" )&\
                                                                            ( col("lab_result")>=0.1)& ( col("lab_result")<0.25 ), '0.1-0.25(可能感染，不建议抗生素)' )\
                                                                   .when( ( col("std_subject")=="降钙素原") &( col("result_type")=="numbers_only" )&\
                                                                            ( col("lab_result")>=0.25)& ( col("lab_result")<=0.5 ), '0.25-0.5(局部感染，建议抗生素)' )\
                                                                    .when( ( col("std_subject")=="降钙素原") &( col("result_type")=="numbers_only" )&\
                                                                            ( col("lab_result")>0.5), '>0.5(严重感染及脓毒症)' )\
                                                                    .when( ( col("std_subject")=="降钙素原") &( col("result_type")=="character"), '>0.5(严重感染及脓毒症)' )\
 
                                                                   .when( ( col("std_subject")=="白细胞计数") &( col("result_type")=="numbers_only")&\
                                                                             ( col("lab_result")>9.3 ), 'H' )\
                                                                   .when( ( col("std_subject")=="白细胞计数") &( col("result_type")=="numbers_only")&\
                                                                             ( col("lab_result")<=9.3 ), 'L' )\
                                                                   .when( ( col("std_subject")=="白细胞计数") &( col("result_type")=="character"), 'L' )\
                                               
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="numbers_only")&\
                                                                             ( col("UNITS").rlike("d") )&( col("lab_result")>=0.8), 'H' )\
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="numbers_only")&\
                                                                             ( col("UNITS").rlike("d") )&( col("lab_result")<0.8), 'L' )\
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="numbers_only")&\
                                                                             ( ~col("UNITS").rlike("d") )&( col("lab_result")>=0.8), 'H' )\
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="numbers_only")&\
                                                                             ( ~col("UNITS").rlike("d") )&( col("lab_result")>=0.8), 'L' )\
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="character")&\
                                                                            ( col("UNITS").rlike("mg/dl") ), 'L' )\
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="character")&\
                                                                            ( ~col("UNITS").rlike("d") )&( col("lab_result").rlike("\\<|\\＜") ), 'L' )\
                                                                   .when( ( col("std_subject")=="C反应蛋白") &( col("result_type")=="character")&\
                                                                            ( ~col("UNITS").rlike("d") ), 'H' )\
                                                                   .when( ( col("std_subject")=="病原菌检测") &( col("lab_result").rlike("可疑|弱阳") ), '弱阳性' )\
                                                                   .when( ( col("std_subject")=="病原菌检测") &( col("lab_result").rlike("阳|\\+|未|无") ), '阳性' )\
                                                                   .when( ( col("std_subject")=="病原菌检测") &( col("lab_result").rlike("未|无|阴|\\-") ), '阴性' )\
                                                                   .when( ( col("std_subject")=="病原菌检测") &( col("pathogenic_type")=="肺炎支原体") &\
                                                                         ( col("value_range").rlike("\\<1\\:40") ) & ( col("result_type")=="numbers_only")&\
                                                                                (col("lab_result")<0.025), '阴性' )\
                                                                   .when( ( col("std_subject")=="病原菌检测") &( col("pathogenic_type")=="肺炎支原体") &\
                                                                         ( col("value_range").rlike("\\<1\\:40") ) & ( col("result_type")=="numbers_only")&\
                                                                                (col("lab_result")>=0.025), '阳性' )\
                                                                   .when( ( col("std_subject")=="病原菌检测") &( col("pathogenic_type")=="肺炎支原体")& ( col("result_type")=="character"), '阴性' )\
                                                                   .otherwise("未知")
                                              )

df_raw_detection_m = df_raw_detection.filter( ( col("std_result").isNotNull() )& ( col("pathogenic_type").isNotNull())|( col("pathogenic_type")=="肺炎支原体") )

# df_raw_detection.show(30)                                               

In [ ]:

#result_unit = UNITS

data_delivery <- data_delivery %>% 
  mutate(std_result = case_when(std_subject == '降钙素原' & result_type == 'numbers_only' & lab_result < 0.1 ~ '<0.1(无感染)',
                                std_subject == '降钙素原' & result_type == 'numbers_only' & lab_result >= 0.1 & lab_result < 0.25 ~ '0.1-0.25(可能感染，不建议抗生素)',
                                std_subject == '降钙素原' & result_type == 'numbers_only' & lab_result >= 0.25 & lab_result <= 0.5 ~ '0.25-0.5(局部感染，建议抗生素)',
                                std_subject == '降钙素原' & result_type == 'numbers_only' & lab_result > 0.5 ~ '>0.5(严重感染及脓毒症)',
                                std_subject == '降钙素原' & result_type == 'character' ~ '>0.5(严重感染及脓毒症)',
                                std_subject == '白细胞计数' & result_type == 'numbers_only' & lab_result > 9.3 ~ 'H',
                                std_subject == '白细胞计数' & result_type == 'numbers_only' & lab_result <= 9.3 ~ 'L',
                                std_subject == '白细胞计数' & result_type == 'character' ~ 'L',
                                std_subject == 'C反应蛋白' & result_type == 'numbers_only' & grepl('d', result_unit) & lab_result >= 0.8 ~ 'H',
                                std_subject == 'C反应蛋白' & result_type == 'numbers_only' & grepl('d', result_unit) & lab_result < 0.8 ~ 'L',
                                std_subject == 'C反应蛋白' & result_type == 'numbers_only' & !grepl('d', result_unit) & lab_result >= 8 ~ 'H',
                                std_subject == 'C反应蛋白' & result_type == 'numbers_only' & !grepl('d', result_unit) & lab_result < 8 ~ 'L',
                                std_subject == 'C反应蛋白' & result_type == 'character' & result_unit == 'mg/dl' ~ 'L', 
                                std_subject == 'C反应蛋白' & result_type == 'character' & !grepl('d', result_unit) & grepl('\\<|\\＜', lab_result) ~ 'L',
                                std_subject == 'C反应蛋白' & result_type == 'character' & !grepl('d', result_unit) ~ 'H',
                                std_subject == '病原菌检测' & grepl('可疑|弱阳', lab_result) ~ '弱阳性', 
                                std_subject == '病原菌检测' & grepl('阳|\\+|未|无', lab_result) ~ '阳性',
                                std_subject == '病原菌检测' & grepl('未|无|阴|\\-', lab_result) ~ '阴性',
                                std_subject == '病原菌检测' & pathogenic_type == '肺炎支原体' & grepl('\\<1\\:40', value_range) & as.numeric(lab_result) < 0.025 ~ '阴性',
                                std_subject == '病原菌检测' & pathogenic_type == '肺炎支原体' & grepl('\\<1\\:40', value_range) & as.numeric(lab_result) >= 0.025 ~ '阳性',
                                std_subject == '病原菌检测' & pathogenic_type == '肺炎支原体' & result_type == 'character' ~ '阴性'

                                )
         )

data_delivery_m <- data_delivery %>% filter((!is.na(std_result) & !is.na(pathogenic_type)) | pathogenic_type == '肺炎支原体')
#write.xlsx(data_delivery, 'Desktop/浪潮/检测返还_0527.xlsx')
write.xlsx(data_delivery_m, 'Desktop/浪潮/检测返还_0528.xlsx')

In [ ]:
# %%

# ##将数据标注区分 检测前 和 检测后
# print( df_raw_detection.count(), df_patient_std.count() )

# df_temp_detection = df_raw_detection.select(["PATIENT_ID", "VISIT_ID", "RESULTS_RPT_DATE_TIME_STD"  ] )\
#                                         .withColumnRenamed("VISIT_ID", "就诊序号") \
#                                         .withColumnRenamed("PATIENT_ID", "患者ID") \
#                                         .withColumnRenamed("RESULTS_RPT_DATE_TIME_STD", "标准处方日期") \
#                                         .withColumn("检测日期", col("标准处方日期").cast("int")  )
# ## 因为每个病人一天会有多个检测,所以需要去重
# df_temp_detection = df_temp_detection.distinct()
# # 就诊序号有空值存在
# df_temp_detection = df_temp_detection.where( col("就诊序号").isNotNull() )
# # 检测日期也有空值存在
# # print( df_temp_detection.where(  Func.isnull(col("检测日期")) ).count()   )
# df_detection_min_date = df_temp_detection.withColumn("最小检测日期", Func.min( col("检测日期") ).over(
#                                             Window.partitionBy("患者ID", "就诊序号") ) )



# df_raw_patient = df_patient_std.withColumn("标准处方日期", col("标准处方日期").cast("int") )

# ##
# df_temp_patient = df_patient_std.join(df_detection_min_date,  on= ["患者ID", "就诊序号", "标准处方日期"], how="left")


# ## 此处以最小检测日期 区分检测前和 检测后
# df_temp_patient = df_temp_patient.withColumn("数据内容", Func.when( col("标准处方日期") < col("最小检测日期"), "检测前结果" )\
#                                                   .when( col("标准处方日期") > col("最小检测日期"), "检测后结果" )\
#                                                   .otherwise("未知") )
# print( df_temp_patient.count() )
